In [1]:
import pandas as pd
import investpy
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#AESGENER
#COPEC
#ENAEX
#OROBLANCO

In [3]:
#Read historical file
cl_hist_stocks = pd.read_csv(r'D:\Anaconda Files\Stocks\CL Stocks\Stocks_CL_coma.csv',  decimal=',' ,sep='\t', encoding='utf-8')
cl_hist_stocks['Date'] = pd.to_datetime(cl_hist_stocks['Date'])
cl_hist_stocks = cl_hist_stocks[['Date', 'Open', 'High', 'Low', 'Close',  'Volume', 'Currency', 'Ticker']]

In [4]:
from datetime import date, datetime, timedelta

#Datetime variables
last_15_days = date.today() - timedelta(days=15)
hoy          = date.today()

#last_365_days = date.today() - timedelta(days=500)

#String Variables
inicio = last_15_days.strftime("%d/%m/%Y")
hoy    = hoy.strftime("%d/%m/%Y")

In [5]:
#Adding Moving Averages
cl_hist_stocks['10_SMA']  = cl_hist_stocks.groupby('Ticker')['Close'].transform(lambda x: x.rolling(10, 1).mean())

cl_hist_stocks['50_SMA']  = cl_hist_stocks.groupby('Ticker')['Close'].transform(lambda x: x.rolling(50, 1).mean())
cl_hist_stocks['150_SMA'] = cl_hist_stocks.groupby('Ticker')['Close'].transform(lambda x: x.rolling(150, 1).mean())
cl_hist_stocks['200_SMA'] = cl_hist_stocks.groupby('Ticker')['Close'].transform(lambda x: x.rolling(200, 1).mean())

#Adding 52-week high
cl_hist_stocks['7D_HIGH']  =   cl_hist_stocks.groupby('Ticker')['Close'].transform(lambda x: x.rolling(7, 1).max())
cl_hist_stocks['7D_LOW']  =   cl_hist_stocks.groupby('Ticker')['Close'].transform(lambda x: x.rolling(7, 1).min())
cl_hist_stocks['14D_HIGH']  =   cl_hist_stocks.groupby('Ticker')['Close'].transform(lambda x: x.rolling(14, 1).max())
cl_hist_stocks['14D_LOW']  =   cl_hist_stocks.groupby('Ticker')['Close'].transform(lambda x: x.rolling(14, 1).min())

cl_hist_stocks['52W_HIGH'] =   cl_hist_stocks.groupby('Ticker')['Close'].transform(lambda x: x.rolling(200, 1).max())

#Adding GR
cl_hist_stocks['W52_Growth'] =  (cl_hist_stocks['Close'] / cl_hist_stocks.groupby('Ticker')['Close'].shift(+253) ) - 1


#Adding 7day highest volume
cl_hist_stocks['20D_SMA_VOL'] =   cl_hist_stocks.groupby('Ticker')['Volume'].transform(lambda x: x.rolling(15, 1).mean())
cl_hist_stocks['7D_HIGH_VOL'] =   cl_hist_stocks.groupby('Ticker')['Volume'].transform(lambda x: x.rolling(7, 1).max())
cl_hist_stocks['20D_SMA_VOL'] =   cl_hist_stocks['20D_SMA_VOL'].astype(int)

#Adding Trading Ranking
cl_hist_stocks['Traded_Amount']  =  cl_hist_stocks['Close'] * cl_hist_stocks['Volume']
cl_hist_stocks['Trading_Rank']   =  cl_hist_stocks.groupby('Date')['Traded_Amount'].rank(ascending=False)
cl_hist_stocks['7D_HIGH_TRADE']  =  cl_hist_stocks.groupby('Ticker')['Volume'].transform(lambda x: x.rolling(7, 1).max())
cl_hist_stocks.drop(['Traded_Amount'], axis=1)

cl_hist_stocks['10D_TRADE_RANK'] =   cl_hist_stocks.groupby('Ticker')['Trading_Rank'].transform(lambda x: x.rolling(10, 1).mean())
cl_hist_stocks['10D_TRADE_RANK'] = cl_hist_stocks['10D_TRADE_RANK'].astype(int)

In [6]:
#Adding Percentile for latest date

max_date = max(cl_hist_stocks["Date"])
Weekday = datetime.weekday(max_date)
print(Weekday)
offset_days = int(2)
if( Weekday == 0):
  offset_days = int(3)
if( Weekday == 1):
  offset_days = int(1)

time1 = datetime.today().strftime("%Y-%m-%d")


cl_hist_stocks_q = cl_hist_stocks[cl_hist_stocks["Date"]  ==  max_date - timedelta(days=offset_days) ]
#cl_hist_stocks_q = cl_hist_stocks[cl_hist_stocks["Date"]  == '2021-04-07']
Weekday = datetime.weekday( date.today())

q10 = cl_hist_stocks_q.W52_Growth.quantile(0.1)
q20 = cl_hist_stocks_q.W52_Growth.quantile(0.2)
q30 = cl_hist_stocks_q.W52_Growth.quantile(0.3)
q40 = cl_hist_stocks_q.W52_Growth.quantile(0.4)
q50 = cl_hist_stocks_q.W52_Growth.quantile(0.5)
q60 = cl_hist_stocks_q.W52_Growth.quantile(0.6)
q65 = cl_hist_stocks_q.W52_Growth.quantile(0.65)
q70 = cl_hist_stocks_q.W52_Growth.quantile(0.7)
q75 = cl_hist_stocks_q.W52_Growth.quantile(0.75)
q80 = cl_hist_stocks_q.W52_Growth.quantile(0.8)
q85 = cl_hist_stocks_q.W52_Growth.quantile(0.85)
q90 = cl_hist_stocks_q.W52_Growth.quantile(0.9)
q95 = cl_hist_stocks_q.W52_Growth.quantile(0.95)
q99 = cl_hist_stocks_q.W52_Growth.quantile(1)

conditions = [
(cl_hist_stocks_q['W52_Growth'] <= q10),
(cl_hist_stocks_q['W52_Growth'] > q10) & (cl_hist_stocks_q['W52_Growth'] <= q20),   
(cl_hist_stocks_q['W52_Growth'] > q20) & (cl_hist_stocks_q['W52_Growth'] <= q30),    
(cl_hist_stocks_q['W52_Growth'] > q30) & (cl_hist_stocks_q['W52_Growth'] <= q40),
(cl_hist_stocks_q['W52_Growth'] > q40) & (cl_hist_stocks_q['W52_Growth'] <= q50),
(cl_hist_stocks_q['W52_Growth'] > q50) & (cl_hist_stocks_q['W52_Growth'] <= q60),
(cl_hist_stocks_q['W52_Growth'] > q60) & (cl_hist_stocks_q['W52_Growth'] <= q65),
(cl_hist_stocks_q['W52_Growth'] > q65) & (cl_hist_stocks_q['W52_Growth'] <= q70),

(cl_hist_stocks_q['W52_Growth'] > q70) & (cl_hist_stocks_q['W52_Growth'] <= q75),
(cl_hist_stocks_q['W52_Growth'] > q75) & (cl_hist_stocks_q['W52_Growth'] <= q80),
(cl_hist_stocks_q['W52_Growth'] > q80) & (cl_hist_stocks_q['W52_Growth'] <= q85),
(cl_hist_stocks_q['W52_Growth'] > q85) & (cl_hist_stocks_q['W52_Growth'] <= q90),
(cl_hist_stocks_q['W52_Growth'] > q90) & (cl_hist_stocks_q['W52_Growth'] <= q95),
(cl_hist_stocks_q['W52_Growth'] > q95) & (cl_hist_stocks_q['W52_Growth'] <= q99)
]

values = [10, 20, 30, 40 ,50, 60, 65, 70, 75, 80, 85, 90, 95, 100]

cl_hist_stocks_q['RS_Latest'] = np.select(conditions, values)
cl_hist_stocks_q2 = cl_hist_stocks_q[['Ticker', 'W52_Growth', 'RS_Latest']]

4


D:\anaconda3\lib\site-packages\ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
#Adding Percentile for two weeks back

max_date = max(cl_hist_stocks["Date"])
Weekday = datetime.weekday(max_date)
offset_days = int(14+2)
if( Weekday == 0):
  offset_days = int(14+3)
if( Weekday == 1):
  offset_days = int(14+1)

time1 = datetime.today().strftime("%Y-%m-%d")

last_2_weeks = max_date - timedelta(days=14)  
last_2_weeks =  max_date - timedelta(days=offset_days) 
cl_hist_stocks_q = cl_hist_stocks[cl_hist_stocks["Date"]  == last_2_weeks]

#pd.to_datetime(last_2_years))


q10 = cl_hist_stocks_q.W52_Growth.quantile(0.1)
q20 = cl_hist_stocks_q.W52_Growth.quantile(0.2)
q30 = cl_hist_stocks_q.W52_Growth.quantile(0.3)
q40 = cl_hist_stocks_q.W52_Growth.quantile(0.4)
q50 = cl_hist_stocks_q.W52_Growth.quantile(0.5)
q60 = cl_hist_stocks_q.W52_Growth.quantile(0.6)
q65 = cl_hist_stocks_q.W52_Growth.quantile(0.65)
q70 = cl_hist_stocks_q.W52_Growth.quantile(0.7)
q75 = cl_hist_stocks_q.W52_Growth.quantile(0.75)
q80 = cl_hist_stocks_q.W52_Growth.quantile(0.8)
q85 = cl_hist_stocks_q.W52_Growth.quantile(0.85)
q90 = cl_hist_stocks_q.W52_Growth.quantile(0.9)
q95 = cl_hist_stocks_q.W52_Growth.quantile(0.95)
q99 = cl_hist_stocks_q.W52_Growth.quantile(1)

conditions = [
(cl_hist_stocks_q['W52_Growth'] <= q10),
(cl_hist_stocks_q['W52_Growth'] > q10) & (cl_hist_stocks_q['W52_Growth'] <= q20),   
(cl_hist_stocks_q['W52_Growth'] > q20) & (cl_hist_stocks_q['W52_Growth'] <= q30),    
(cl_hist_stocks_q['W52_Growth'] > q30) & (cl_hist_stocks_q['W52_Growth'] <= q40),
(cl_hist_stocks_q['W52_Growth'] > q40) & (cl_hist_stocks_q['W52_Growth'] <= q50),
(cl_hist_stocks_q['W52_Growth'] > q50) & (cl_hist_stocks_q['W52_Growth'] <= q60),
(cl_hist_stocks_q['W52_Growth'] > q60) & (cl_hist_stocks_q['W52_Growth'] <= q65),
(cl_hist_stocks_q['W52_Growth'] > q65) & (cl_hist_stocks_q['W52_Growth'] <= q70),

(cl_hist_stocks_q['W52_Growth'] > q70) & (cl_hist_stocks_q['W52_Growth'] <= q75),
(cl_hist_stocks_q['W52_Growth'] > q75) & (cl_hist_stocks_q['W52_Growth'] <= q80),
(cl_hist_stocks_q['W52_Growth'] > q80) & (cl_hist_stocks_q['W52_Growth'] <= q85),
(cl_hist_stocks_q['W52_Growth'] > q85) & (cl_hist_stocks_q['W52_Growth'] <= q90),
(cl_hist_stocks_q['W52_Growth'] > q90) & (cl_hist_stocks_q['W52_Growth'] <= q95),
(cl_hist_stocks_q['W52_Growth'] > q95) & (cl_hist_stocks_q['W52_Growth'] <= q99)
]

values = [10, 20, 30, 40 ,50, 60, 65, 70, 75, 80, 85, 90, 95, 100]

cl_hist_stocks_q['RS_2Weeks'] = np.select(conditions, values)
cl_hist_stocks_q3 = cl_hist_stocks_q[['Ticker', 'RS_2Weeks']]

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
cl_hist_stocks_q3

,Ticker,RS_2Weeks
43893,AESGENER,50
43902,AGUASA,10
43909,ALMENDRAL,40
43917,ANDINAA,60
43925,ANDINAB,40
...,...,...
44601,TRICOT,80
44609,V,90
44617,VAPORES,100
44622,VOLCAN,0


In [9]:
#Creating file
#Technical_filter = pd.DataFrame(columns=['Ticker', '200MA', '52Week_High', 'RS'])
Technical_filter = pd.DataFrame(columns=['Ticker', 'MA', '52Week_High', 'Volume_Spike', 'Most_Traded', '10_Vol_MA_Rank', 'Z_7D', 'Z_14D'])

stock_list = cl_hist_stocks.Ticker.unique()
for i in range(len(stock_list)):
      Ticker = str(stock_list[i])
      Bool_MA_200   = 0  
      Bool_52W_High = 0
      Bool_Vol_Peak = 0
      Bool_Most_Traded  = 0
        
      current_price = float(cl_hist_stocks[cl_hist_stocks['Ticker'] == Ticker]["Close"].tail(1))
      MA_200 =        float(cl_hist_stocks[cl_hist_stocks['Ticker'] == Ticker]["200_SMA"].tail(1))     
      MA_150 =        float(cl_hist_stocks[cl_hist_stocks['Ticker'] == Ticker]["150_SMA"].tail(1))         
      MA_50  =        float(cl_hist_stocks[cl_hist_stocks['Ticker'] == Ticker]["50_SMA"].tail(1))   
      #MA_50_7D  =        float(cl_hist_stocks[cl_hist_stocks['Ticker'] == Ticker]["50_SMA"].tail(7))   
    
      a7    =         float(cl_hist_stocks[cl_hist_stocks['Ticker'] == Ticker]["7D_HIGH"].tail(1))       
      a52   =         float(cl_hist_stocks[cl_hist_stocks['Ticker'] == Ticker]["52W_HIGH"].tail(1))   
    
      vol_max_7 =     float(cl_hist_stocks[cl_hist_stocks['Ticker'] == Ticker]["7D_HIGH_VOL"].tail(1)) 
      vol_MA_15 =     float(cl_hist_stocks[cl_hist_stocks['Ticker'] == Ticker]["20D_SMA_VOL"].tail(1))
     
      Traded_Rank =     float(cl_hist_stocks[cl_hist_stocks['Ticker'] == Ticker]["Trading_Rank"].tail(1)) 
      Traded_Rank_MA =  float(cl_hist_stocks[cl_hist_stocks['Ticker'] == Ticker]["10D_TRADE_RANK"].tail(1))  
       
          
        
      try:
       MAX_14 =          float(cl_hist_stocks[cl_hist_stocks['Ticker'] == Ticker]["14D_HIGH"].iloc[-7])  
      except:
       MAX_14 =          float(cl_hist_stocks[cl_hist_stocks['Ticker'] == Ticker]["14D_HIGH"].iloc[1])  
    
      MIN_7 =           float(cl_hist_stocks[cl_hist_stocks['Ticker'] == Ticker]["7D_LOW"].tail(1))       
    
      if (current_price > MA_50) & (MA_50 > MA_150) & (MA_150 > MA_200):
        Bool_MA_200 = 1
        
      if a7 >= a52:
        Bool_52W_High = 1
        
      if vol_max_7 > vol_MA_15 * 2:
        Bool_Vol_Peak  = 1
        
      if Traded_Rank <= 12:
        Bool_Most_Traded = 1
        
      Z_7  = MIN_7  / MA_50
      Z_14 = MAX_14 / MA_50     
      
        
     
        
      #print(Ticker + ' 7days '+ str(a7) + '  52d ' + str(a52) )
      Technical_filter = Technical_filter.append({'Ticker':      Ticker, 
                                                  'MA'    :      Bool_MA_200, 
                                                  '52Week_High': Bool_52W_High,
                                                  'Volume_Spike':Bool_Vol_Peak,
                                                  'Most_Traded': Bool_Most_Traded,
                                                  '10_Vol_MA_Rank':  Traded_Rank_MA,
                                                  'Z_7D' : Z_7,
                                                  'Z_14D': Z_14                                                 
                                                
                                                  
                                                 }, ignore_index=True)     


In [10]:
#Merge
TF = pd.merge(Technical_filter, cl_hist_stocks_q2, how='left')
Technical_filter = pd.merge(TF, cl_hist_stocks_q3, how='left')


In [11]:
TF

,Ticker,MA,52Week_High,Volume_Spike,Most_Traded,10_Vol_MA_Rank,Z_7D,Z_14D,W52_Growth,RS_Latest
0,AESGENER,0,0,1,0,21.0,0.962977,0.997882,-0.166213,50.0
1,AGUASA,0,0,1,0,19.0,0.971406,0.978708,-0.417045,10.0
2,AGUNSA,0,0,1,0,89.0,0.998240,0.998240,NaN,NaN
3,ALMENDRAL,0,0,0,0,57.0,0.899956,0.972038,NaN,NaN
4,ANDINAA,0,0,1,0,44.0,1.024322,1.024322,0.032899,70.0
...,...,...,...,...,...,...,...,...,...,...
129,VSPT,1,1,1,0,81.0,1.008684,1.008684,NaN,NaN
130,VZ,0,0,1,0,86.0,0.960482,1.024232,NaN,NaN
131,WAT,0,0,1,0,54.0,0.884808,1.004199,-0.260332,30.0
132,WFC,0,0,0,0,87.0,1.324061,1.411217,NaN,NaN


In [12]:
Technical_filter[Technical_filter['Most_Traded'] == 1 ]

,Ticker,MA,52Week_High,Volume_Spike,Most_Traded,10_Vol_MA_Rank,Z_7D,Z_14D,W52_Growth,RS_Latest,RS_2Weeks
17,BSANTANDER,0,0,1,1,7.0,0.973842,1.009008,0.119632,80.0,70.0
21,CAP,0,0,0,1,8.0,0.974167,1.038356,1.261633,100.0,100.0
22,CCU,1,1,0,1,15.0,1.093756,1.099204,0.432876,90.0,85.0
26,CENCOSUD,0,0,0,1,6.0,0.942041,1.016134,0.034884,70.0,70.0
29,CHILE,0,0,0,1,6.0,0.935480,0.997329,-0.056871,60.0,60.0
32,CMPC,0,0,0,1,12.0,0.905293,1.000649,-0.060046,60.0,60.0
33,COLBUN,0,0,0,1,21.0,0.997401,1.059221,-0.211029,40.0,30.0
36,CONCHATORO,1,0,1,1,18.0,0.999817,1.028713,0.064696,75.0,75.0
62,FALABELLA,0,0,1,1,4.0,0.927434,1.010871,0.142488,80.0,80.0
120,SQMA,0,0,0,1,7.0,1.053921,1.064047,0.752275,100.0,90.0


In [13]:
Technical_filter.to_excel(r'D:\Anaconda Files\Stocks\CL Stocks\Technical_filter.xlsx')

#Technical_filter.to_csv(r'C:\Users\Francisco Montt\Dropbox\Emprendimientos\Inversiones\Technical_filter.csv',  decimal=',' , sep='\t', encoding='utf-8')
#df_total.to_excel('combined_file.xlsx')
Technical_filter.to_excel(r'C:\Users\Francisco Montt\Dropbox\Emprendimientos\Inversiones\Technical_Filter.xlsx')

In [14]:
cl_hist_stocks = cl_hist_stocks[cl_hist_stocks["Date"]  >= '2020-03-12']
cl_hist_stocks.to_csv(r'D:\Anaconda Files\Stocks\CL Stocks\Rolling_Analysis.csv',  decimal=',' , sep='\t', encoding='utf-8')

In [15]:
#candidates = cl_hist_stocks2.Ticker.unique()
#candidates
cl_hist_stocks[cl_hist_stocks["Date"]  >= '2019-06-01']

,Date,Open,High,Low,Close,Volume,Currency,Ticker,10_SMA,50_SMA,...,14D_HIGH,14D_LOW,52W_HIGH,W52_Growth,20D_SMA_VOL,7D_HIGH_VOL,Traded_Amount,Trading_Rank,7D_HIGH_TRADE,10D_TRADE_RANK
139,2020-03-12,120.23,122.46,116.97,117.41,15147594,CLP,AESGENER,117.360,134.1308,...,133.01,110.48,155.94,NaN,12586862,24387692,1.778479e+09,18.0,24387692,16
140,2020-03-13,122.06,123.84,118.33,122.00,9117755,CLP,AESGENER,118.512,133.5602,...,133.01,110.48,155.94,NaN,13118992,24387692,1.112366e+09,19.0,24387692,16
141,2020-03-16,119.16,119.16,105.49,105.49,20775040,CLP,AESGENER,117.769,132.6650,...,133.01,105.49,155.94,NaN,14262590,20997424,2.191559e+09,12.0,20997424,16
142,2020-03-17,110.08,110.99,104.56,105.02,28270828,CLP,AESGENER,116.988,131.7298,...,130.59,105.02,155.94,NaN,15884371,28270828,2.969002e+09,15.0,28270828,16
143,2020-03-18,104.19,104.19,93.56,94.48,29718114,CLP,AESGENER,114.786,130.6246,...,123.84,94.48,155.94,NaN,17690039,29718114,2.807767e+09,16.0,29718114,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45483,2021-08-02,366.86,367.00,366.86,366.93,18185,CLP,ZOFRI,375.649,398.5166,...,382.80,366.86,519.37,-0.291860,12516,45078,6.672622e+06,56.0,45078,65
45484,2021-08-03,367.00,367.00,367.00,367.00,121416,CLP,ZOFRI,374.069,396.8566,...,382.80,366.86,519.37,-0.266000,20583,121416,4.455967e+07,43.0,121416,63
45485,2021-08-04,367.00,367.00,367.00,367.00,1000,CLP,ZOFRI,372.489,395.1966,...,382.80,366.86,519.37,-0.266000,19107,121416,3.670000e+05,73.0,121416,64
45486,2021-08-05,367.10,367.10,367.10,367.00,5300,CLP,ZOFRI,370.909,393.5366,...,382.80,366.86,519.37,-0.274259,17173,121416,1.945100e+06,71.0,121416,64


In [16]:
    ticker = candidates[0]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

NameError: name 'candidates' is not defined

In [ ]:
df.tail(3)

In [ ]:
    print(cl_hist_stocks2.Ticker.unique()[1])
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == cl_hist_stocks2.Ticker.unique()[1]]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
df.tail(3)

In [ ]:
    print(cl_hist_stocks2.Ticker.unique()[2])
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == cl_hist_stocks2.Ticker.unique()[2]]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    print(cl_hist_stocks2.Ticker.unique()[3])
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == cl_hist_stocks2.Ticker.unique()[3]]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    print(cl_hist_stocks2.Ticker.unique()[4])
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == cl_hist_stocks2.Ticker.unique()[4]]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    print(cl_hist_stocks2.Ticker.unique()[5])
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == cl_hist_stocks2.Ticker.unique()[5]]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    print(cl_hist_stocks2.Ticker.unique()[6])
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == cl_hist_stocks2.Ticker.unique()[6]]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[7]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[8]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[9]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[10]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[11]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[12]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[13]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[14]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[15]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[16]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[17]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[18]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[19]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[20]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[21]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[22]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = 'ENELAM'
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[24]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[25]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[26]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[27]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[28]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[29]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[30]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[31]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[32]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[33]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
    ticker = cl_hist_stocks2.Ticker.unique()[34]
    print(ticker)
    df = cl_hist_stocks[cl_hist_stocks["Ticker"]  == ticker]
    fig = plt.figure(figsize=(20,8))
    sns.lineplot('Date', 'Open', data=df)
    sns.lineplot("Date","50_SMA", data=df, legend='brief')
    sns.lineplot("Date","150_SMA", data=df)
    sns.lineplot("Date","200_SMA", data=df)

In [ ]:
cl_hist_stocks_dummy

In [ ]:
cl_hist_stocks_dummy[cl_hist_stocks_dummy['Ticker'] == 'CENCOSUD' ]

In [ ]:
      = cl_hist_stocks_dummy2['Open']

In [ ]:
    stock_list = ['AESGENER']
    #stock_list = cl_hist_stocks.Ticker.unique()
    concat = cl_hist_stocks[cl_hist_stocks['Ticker'] == 'Nada' ]

#for i in range(len(stock_list)):
   
    cl_hist_stocks_dummy = cl_hist_stocks[cl_hist_stocks['Ticker'] == 'AESGENER' ]
                                      
    cl_hist_stocks_dummy2 = cl_hist_stocks_dummy[cl_hist_stocks_dummy["Date"]  == pd.to_datetime(last_15_days) ]

   # cl_hist_stocks_dummy['52_Week']  =  cl_hist_stocks_dummy2['Open'].mean()
    
    #concat = pd.concat([concat, cl_hist_stocks_dummy], axis=0)
    
#cl_hist_stocks = concat

In [ ]:
cl_hist_stocks_dummy = cl_hist_stocks[cl_hist_stocks['Ticker'] == 'AESGENER' ]
                                      
cl_hist_stocks_dummy2 = cl_hist_stocks_dummy[cl_hist_stocks_dummy["Date"]  == pd.to_datetime(last_15_days) ]

In [ ]:
cl_hist_stocks_dummy2

In [ ]:
cl_hist_stocks_dummy[cl_hist_stocks_dummy["Date"]  == pd.to_datetime(last_15_days) ]

In [ ]:
cl_hist_stocks['10_SMA'].max()

In [ ]:
 cl_hist_stocks_dummy[cl_hist_stocks_dummy["Date"]  >= pd.to_datetime(last_365_days) ]

In [ ]:

    cl_hist_stocks_dummy = cl_hist_stocks[cl_hist_stocks['Ticker'] == 'AESGENER' ]
                                      


    
   # cl_hist_stocks_dummy['52_Week']  =  cl_hist_stocks_dummy2['Open'].mean()
    
   
    

In [ ]:
cl_hist_stocks_dummy[cl_hist_stocks_dummy["Date"]  >= pd.to_datetime(last_365_days) ]

In [ ]:
a=1

In [ ]:
b=2

In [ ]:
a+b

In [ ]:
ade

In [ ]:
#Adding 52-week prices

stock_list = cl_hist_stocks.Ticker.unique()
concat = cl_hist_stocks[cl_hist_stocks['Ticker'] == 'Nada' ]

for i in range(len(stock_list)):
   
    cl_hist_stocks_dummy = cl_hist_stocks[cl_hist_stocks['Ticker'] == stock_list[i] ]
                                      
    cl_hist_stocks_dummy2 = cl_hist_stocks_dummy[cl_hist_stocks_dummy["Date"]  == pd.to_datetime(last_365_days) ]

    
    cl_hist_stocks_dummy['52_Week']  =  cl_hist_stocks_dummy2['Open'].mean()
    
   
    cl_hist_stocks_dummy2 = cl_hist_stocks_dummy[cl_hist_stocks_dummy["Date"]  >= pd.to_datetime(last_365_days) ]
      
    cl_hist_stocks_dummy['52_Week_High']  =  cl_hist_stocks_dummy2['Open'].max()    
    
    
    concat = pd.concat([concat, cl_hist_stocks_dummy], axis=0)
    
cl_hist_stocks = concat
    
    

In [ ]:
#Satisfying condition 50MA > 200MA
cl_hist_stocks2 = cl_hist_stocks.copy() 
cl_hist_stocks2 = cl_hist_stocks2.loc[(cl_hist_stocks2.Date > pd.to_datetime(last_15_days))]


#cl_hist_stocks2 = cl_hist_stocks2[cl_hist_stocks2["Open"]  < cl_hist_stocks2["200_SMA"] * 1.1]
#cl_hist_stocks2 = cl_hist_stocks2[cl_hist_stocks2["Open"]  > cl_hist_stocks2["150_SMA"]]

cl_hist_stocks2 = cl_hist_stocks2[cl_hist_stocks2["50_SMA"]  > cl_hist_stocks2["150_SMA"]]
cl_hist_stocks2 = cl_hist_stocks2[cl_hist_stocks2["150_SMA"]  > cl_hist_stocks2["200_SMA"]]

#cl_hist_stocks2 = cl_hist_stocks2[cl_hist_stocks2["50_SMA"]  < cl_hist_stocks2["200_SMA"] * 1.1]



#cl_hist_stocks2 = cl_hist_stocks2[cl_hist_stocks2["Open"]  > cl_hist_stocks2["52_Week"]]
#cl_hist_stocks2 = cl_hist_stocks2[cl_hist_stocks2["Open"]  < cl_hist_stocks2["52_Week"]* 1.1]
#cl_hist_stocks2 = cl_hist_stocks2[cl_hist_stocks2["Open"]  > cl_hist_stocks2["52_Week_High"]]



#cl_hist_stocks = cl_hist_stocks['52_Week']

In [ ]:
for i in range(2):
    q = 'q' + str(i)
    q = cl_hist_stocks

In [ ]:
q = cl_hist_stocks

In [ ]:
q = 'q' + str(i)

In [ ]:
cl_hist_stocks.nlargest(3,'Date')

In [ ]:
cl_hist_stocks[cl_hist_stocks["Date"]  > '2021-04-06']

In [ ]:
max(cl_hist_stocks.Date)

In [ ]:
cl_hist_stocks_q3

In [ ]:
cl_hist_stocks[cl_hist_stocks["Date"]  == last_2_weeks]

In [ ]:
max_date

In [ ]:
cl_hist_stocks['Date'].count()

In [ ]:
cl_hist_stocks.groupby('Date')['Ticker'].count()

In [ ]:
datetime.weekday( date.today() + timedelta(days=0) )

In [ ]:
date.today()

In [ ]:
Weekday = datetime.weekday( date.today())


In [ ]:
Weekday

In [ ]:
format = "%d/%m/%Y"
#format datetime using strftime() 
time1 = datetime.today().strftime("%d/%m/%Y")

In [ ]:
time1


In [ ]:
cl_hist_stocks

In [ ]:
time1 - timedelta(days=offset_days)

In [ ]:
time1 = datetime.today().strftime("%Y-%m-%d")

In [ ]:
time1

In [ ]:
dt_object = datetime.strptime(time1, "%Y-%m-%d")

In [ ]:
dt_object

In [ ]:
pd.to_datetime(time1))

In [ ]:
max_date

In [ ]:
cl_hist_stocks.iloc[-2
                   ]